In [ ]:
!pip install -q kaggle gdown
import os

os.makedirs('/root/.kaggle', exist_ok=True)
!echo '{"username":"venkatasaiprakashye","key":"df2bf512defd565c9b99bee7e383a463"}' > /root/.kaggle/kaggle.json
!chmod 600 /root/.kaggle/kaggle.json

!kaggle datasets download -d jessicali9530/celeba-dataset -p celeba
!unzip celeba/celeba-dataset.zip -d celeba/
!mkdir -p celeba/train
!mv celeba/img_align_celeba/* celeba/train/
!rm celeba/celeba-dataset.zip

!ls celeba/train | head -n 5

import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image

class CelebADataset(Dataset):
    def __init__(self, transform=None):
        self.image_dir = "celeba/train/img_align_celeba"
        self.transform = transform

        self.image_files = sorted([f for f in os.listdir(self.image_dir) if f.endswith('.jpg')])
        print(f"Found {len(self.image_files)} images")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

def setup_dataloader(batch_size=128, image_size=64, num_workers=2):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    dataset = CelebADataset(transform=transform)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
        drop_last=True
    )

    return dataloader

def test_dataloader():
    import matplotlib.pyplot as plt

    dataloader = setup_dataloader(batch_size=4)

    images = next(iter(dataloader))
    print(f"Batch shape: {images.shape}")

    plt.figure(figsize=(12, 3))
    for i in range(4):
        plt.subplot(1, 4, i+1)
        img = images[i].permute(1, 2, 0) * 0.5 + 0.5
        plt.imshow(img)
        plt.axis('off')
    plt.show()

test_dataloader()

Dataset URL: https://www.kaggle.com/datasets/jessicali9530/celeba-dataset
License(s): other
 15% 208M/1.33G [00:11<01:02, 19.3MB/s]
User cancelled operation
Archive:  celeba/celeba-dataset.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of celeba/celeba-dataset.zip or
        celeba/celeba-dataset.zip.zip, and cannot find celeba/celeba-dataset.zip.ZIP, period.
mv: cannot stat 'celeba/img_align_celeba/*': No such file or directory


FileNotFoundError: [Errno 2] No such file or directory: 'celeba/train/img_align_celeba'

In [ ]:
import os
import kagglehub
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import matplotlib.pyplot as plt

print("Checking directory contents:")
!ls /root/.cache/kagglehub/datasets/jessicali9530/celeba-dataset/versions/2

class CelebADataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.transform = transform
        self.image_list = []
        for root, _, files in os.walk(root_dir):
            for file in files:
                if file.endswith('.jpg'):
                    self.image_list.append(os.path.join(root, file))
        print(f"Found {len(self.image_list)} images")

    def __len__(self):
        return len(self.image_list)

    def __getitem__(self, idx):
        img_path = self.image_list[idx]
        image = Image.open(img_path)

        if self.transform:
            image = self.transform(image)

        return image, 0

def setup_data(batch_size=128):
    transform = transforms.Compose([
        transforms.Resize(64),
        transforms.CenterCrop(64),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    dataset = CelebADataset(
        root_dir='celeba/train/img_align_celeba',
        transform=transform
    )

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True
    )

    return dataloader

def test_data_loading():
    try:
        dataloader = setup_data(batch_size=4)
        print(f"Dataset size: {len(dataloader.dataset)}")

        images, _ = next(iter(dataloader))
        print(f"Batch shape: {images.shape}")

        plt.figure(figsize=(12, 3))
        for i in range(4):
            plt.subplot(1, 4, i+1)
            plt.imshow(images[i].permute(1, 2, 0).numpy() * 0.5 + 0.5)
            plt.axis('off')
        plt.show()

        return "Data loading successful!"

    except Exception as e:
        print(f"Error loading data: {str(e)}")
        return "Data loading failed"


print(test_data_loading())

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import ipywidgets as widgets
from IPython.display import display

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

class Generator(nn.Module):
    def __init__(self, latent_dim=100, channels=3):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim

        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),

            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),

            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),

            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),

            nn.ConvTranspose2d(64, channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = x.view(-1, self.latent_dim, 1, 1)
        return self.main(x)

class Discriminator(nn.Module):
    def __init__(self, channels=3):
        super(Discriminator, self).__init__()

        self.main = nn.Sequential(
            nn.Conv2d(channels, 64, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(64, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(128, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(256, 512, 4, 2, 1, bias=False),
            nn.BatchNorm2d(512),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Conv2d(512, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.main(x).view(-1, 1).squeeze(1)

# Dataset class with image limit
class CelebADataset(Dataset):
    def __init__(self, transform=None, max_images=30000):
        self.image_dir = "celeba/train/img_align_celeba"
        self.transform = transform

        # Get all image files and limit to max_images
        self.image_files = sorted([f for f in os.listdir(self.image_dir) if f.endswith('.jpg')])[:max_images]
        print(f"Found {len(self.image_files)} images")

    def __len__(self):
        return len(self.image_files)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_files[idx])
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image

def setup_data(batch_size=128, image_size=64, max_images=30000):
    transform = transforms.Compose([
        transforms.Resize(image_size),
        transforms.CenterCrop(image_size),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])

    dataset = CelebADataset(transform=transform, max_images=max_images)

    dataloader = DataLoader(
        dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
        drop_last=True
    )

    return dataloader

def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)

def train_gan(dataloader, num_epochs=100, latent_dim=100):
    os.makedirs('models', exist_ok=True)

    netG = Generator(latent_dim=latent_dim).to(device)
    netD = Discriminator().to(device)

    netG.apply(weights_init)
    netD.apply(weights_init)

    criterion = nn.BCELoss()
    optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

    fixed_noise = torch.randn(64, latent_dim, 1, 1, device=device)

    print("Starting Training Loop...")
    for epoch in range(num_epochs):
        progress_bar = tqdm(enumerate(dataloader), total=len(dataloader))
        for i, real_images in progress_bar:
            batch_size = real_images.size(0)
            real_images = real_images.to(device)

            real_label = torch.ones(batch_size, device=device)
            fake_label = torch.zeros(batch_size, device=device)

            # Train Discriminator
            netD.zero_grad()
            output = netD(real_images)
            errD_real = criterion(output, real_label)
            D_x = output.mean().item()

            # Train with fake images
            noise = torch.randn(batch_size, latent_dim, 1, 1, device=device)
            fake_images = netG(noise)
            output = netD(fake_images.detach())
            errD_fake = criterion(output, fake_label)
            D_G_z1 = output.mean().item()

            errD = errD_real + errD_fake
            errD.backward()
            optimizerD.step()

            # Train Generator
            netG.zero_grad()
            output = netD(fake_images)
            errG = criterion(output, real_label)
            errG.backward()
            D_G_z2 = output.mean().item()
            optimizerG.step()

            progress_bar.set_description(
                f'[{epoch}/{num_epochs-1}][{i}/{len(dataloader)-1}] '
                f'Loss_D: {errD.item():.3f} Loss_G: {errG.item():.3f} '
                f'D(x): {D_x:.3f} D(G(z)): {D_G_z1:.3f}/{D_G_z2:.3f}'
            )

        if (epoch + 1) % 10 == 0:
            with torch.no_grad():
                fake = netG(fixed_noise).detach().cpu()
                plt.figure(figsize=(8, 8))
                plt.axis("off")
                plt.title(f"Fake Images Epoch {epoch+1}")
                plt.imshow(np.transpose(torchvision.utils.make_grid(fake, padding=2, normalize=True), (1,2,0)))
                plt.show()

    torch.save({
        'generator_state_dict': netG.state_dict(),
        'discriminator_state_dict': netD.state_dict(),
        'latent_dim': latent_dim
    }, 'models/image_gen_model.pt')

    print("Training finished! Model saved as 'models/image_gen_model.pt'")
    return netG, netD

class GANInterface:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        checkpoint = torch.load('models/image_gen_model.pt')

        self.generator = Generator(latent_dim=checkpoint['latent_dim']).to(self.device)
        self.generator.load_state_dict(checkpoint['generator_state_dict'])
        self.generator.eval()

        self.latent_dim = checkpoint['latent_dim']
        self.noise = torch.randn(1, self.latent_dim, 1, 1, device=self.device)
        self.setup_widgets()

    def setup_widgets(self):
        self.noise_slider = widgets.FloatSlider(
            value=1.0,
            min=0.0,
            max=2.0,
            step=0.1,
            description='Noise Scale:',
            continuous_update=False
        )

        self.generate_button = widgets.Button(
            description='Generate New Face',
            button_style='primary'
        )

        self.noise_slider.observe(self.on_noise_change, names='value')
        self.generate_button.on_click(self.on_generate_click)

        display(widgets.VBox([
            self.noise_slider,
            self.generate_button
        ]))

        self.generate_image()

    def on_noise_change(self, change):
        self.noise = self.noise * change.new / change.old
        self.generate_image()

    def on_generate_click(self, b):
        self.noise = torch.randn(1, self.latent_dim, 1, 1, device=self.device)
        self.generate_image()

    def generate_image(self):
        with torch.no_grad():
            fake = self.generator(self.noise)
            img = fake[0].cpu()
            img = (img + 1) / 2.0

            plt.figure(figsize=(8, 8))
            plt.imshow(img.permute(1, 2, 0))
            plt.axis('off')
            plt.show()

if __name__ == "__main__":
    try:
        print("Setting up data...")
        dataloader = setup_data(batch_size=128, max_images=30000)

        print("\nStarting GAN training...")
        netG, netD = train_gan(dataloader, num_epochs=100)

        print("\nCreating interface...")
        interface = GANInterface()

    except Exception as e:
        print(f"An error occurred: {str(e)}")

KeyboardInterrupt: 

In [3]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.4/320.4 kB 28.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 106.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 6.1 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [4]:
import torch
import gradio as gr
import numpy as np
from torchvision import transforms
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self, latent_dim=100, channels=3):
        super(Generator, self).__init__()
        self.latent_dim = latent_dim

        self.main = nn.Sequential(
            nn.ConvTranspose2d(latent_dim, 512, 4, 1, 0, bias=False),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.ConvTranspose2d(512, 256, 4, 2, 1, bias=False),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.ConvTranspose2d(256, 128, 4, 2, 1, bias=False),
            nn.BatchNorm2d(128),
            nn.ReLU(True),
            nn.ConvTranspose2d(128, 64, 4, 2, 1, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.ConvTranspose2d(64, channels, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, x):
        x = x.view(-1, self.latent_dim, 1, 1)
        return self.main(x)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load('/content/models/image_gen_model.pt')
generator = Generator(latent_dim=checkpoint['latent_dim']).to(device)
generator.load_state_dict(checkpoint['generator_state_dict'])
generator.eval()

def generate_face(noise_scale, seed):
    torch.manual_seed(seed)

    noise = torch.randn(1, checkpoint['latent_dim'], 1, 1, device=device) * noise_scale

    with torch.no_grad():
        fake = generator(noise)
        img = fake[0].cpu()
        img = (img + 1) / 2.0
        img = img.permute(1, 2, 0).numpy()
        img = (img * 255).astype(np.uint8)

    return img

demo = gr.Interface(
    fn=generate_face,
    inputs=[
        gr.Slider(minimum=0, maximum=2, value=1.0, step=0.1, label="Noise Scale"),
        gr.Slider(minimum=0, maximum=100000, value=42, step=1, label="Random Seed")
    ],
    outputs=gr.Image(label="Generated Face", type="numpy"),
    title="Face Generator",
    description="Generate faces using a GAN trained on CelebA dataset. Adjust the noise scale to control the variation in generated images.",
    examples=[[1.0, 42], [0.5, 123], [1.5, 999]],
    allow_flagging="never"
)

demo.launch(share=True)

<ipython-input-4-b0d535e11f1f>:36: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('/content/models/image_gen_model.pt')
/usr/local/lib/python3.10/dist

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b7a2ac9eaf884f743b.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
